# **openHPI Course: Knowledge Graphs 2023**

## **Week 3: Querying Knowledge Graphs with SPARQL**
### **Notebook 3.1: Wikidata**
---

This is the python notebook for week 3 (Querying Knowledge Graphs with SPARQL) in the openHPI Course **Knowledge Graphs 2023**.

In this colab notebook you will learn how to query the Wikidata Knowledge Graph.

*Please make a copy of this notebook to try out your own adaptions via "File -> Save Copy in Drive"*

### Querying Wikidata with mkwikidata ###

[mkwikidata](https://pypi.org/project/mkwikidata/#description) is a package for running SPARQL queries on Wikidata

We have to install the following packages:

**mkwikidata**

In [ ]:
!pip install mkwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import mkwikidata, pprint
import pandas as pd

#### **Query 1**:  Show all narrated cities of books and literary works and their geo coordination in Wikidata

Following Wikidata classes/instances and properties are required:

[wd:Q571](https://www.wikidata.org/wiki/Q571), [wd:Q7725634](https://www.wikidata.org/wiki/Q7725634), [wd:Q515](https://www.wikidata.org/wiki/Q515),  [wdt:P31](https://www.wikidata.org/wiki/Property:P31), [wdt:P840](https://www.wikidata.org/wiki/Property:P840), [wdt:P625](https://www.wikidata.org/wiki/Property:P625) , [wdt:P279](https://www.wikidata.org/wiki/Property:P279)

In [ ]:
query = """
SELECT ?label ?placelabel ?coord
WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634} #book or literary work
  ?item wdt:P31 ?type . #of type ?type
  ?item rdfs:label ?label FILTER (lang(?label) = "en") .
  ?item wdt:P840 ?place . #narration place ?place
  ?place wdt:P31/wdt:P279* wd:Q515 . # place is an instance or a subclass of city
  ?place wdt:P625 ?coord . #has coordinations ?ccord
  ?place rdfs:label ?placelabel FILTER (lang(?placelabel) = "en") .
}
"""
query_result = mkwikidata.run_query(query, params={ })
query_result

{'head': {'vars': ['label', 'placelabel', 'coord']},
 'results': {'bindings': [{'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(11.254166666 43.771388888)'},
    'label': {'xml:lang': 'en', 'type': 'literal', 'value': 'The Decameron'},
    'placelabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Florence'}},
   {'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(11.254166666 43.771388888)'},
    'label': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'A Room with a View'},
    'placelabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Florence'}},
   {'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(10.992777777 45.438611111)'},
    'label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Das Eckenlied'},
    'placelabel': {'xml:lang': 'en', 'type': 'literal', 'valu

The json_normalize() function is used to convert the JSON string into a DataFrame for a more (human)readable output

In [ ]:
results_df = pd.io.json.json_normalize(query_result['results']['bindings'])
results_df[['label.value', 'placelabel.value', 'coord.value']]

<ipython-input-21-7bbac26bb9e5>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(query_result['results']['bindings'])


,label.value,placelabel.value,coord.value
0,The Decameron,Florence,Point(11.254166666 43.771388888)
1,A Room with a View,Florence,Point(11.254166666 43.771388888)
2,Das Eckenlied,Verona,Point(10.992777777 45.438611111)
3,Romeo and Juliet,Verona,Point(10.992777777 45.438611111)
4,Wounded to Death,Naples,Point(14.25 40.833333333)
...,...,...,...
14187,The Stochastic Man,New York City,Point(-74.0 40.7)
14188,The Naked Face,New York City,Point(-74.0 40.7)
14189,City of Ashes,New York City,Point(-74.0 40.7)
14190,City of Glass,New York City,Point(-74.0 40.7)


Let's [visualize the query](https://query.wikidata.org/#%23defaultView%3AMap%0ASELECT%20%3Flabel%20%3Fplacelabel%20%3Fcoord%0AWHERE%0A%7B%0A%20%20VALUES%20%3Ftype%20%7Bwd%3AQ571%20wd%3AQ7725634%7D%20%20%23%20book%20or%20literary%20work%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%20.%0A%20%20%3Fitem%20rdfs%3Alabel%20%3Flabel%20FILTER%20%28lang%28%3Flabel%29%20%3D%20%22en%22%29%20.%0A%20%20%3Fitem%20wdt%3AP840%20%3Fplace%20.%20%20%23%20narration%20place%20is%20%3Fplace%0A%20%20%3Fplace%20wdt%3AP625%20%3Fcoord%20.%0A%20%20%3Fplace%20rdfs%3Alabel%20%3Fplacelabel%20FILTER%20%28lang%28%3Fplacelabel%29%20%3D%20%22en%22%29%20.%0A%7D%20%0A) result in Wikidata Query Service\!

#### **Query 2**: What is the most common narration country in Wikidata books and literature works? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P17](https://www.wikidata.org/wiki/Property:P17)

In [ ]:
query = """
SELECT ?countrylabel (COUNT(?country) AS ?count)
WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
  ?item wdt:P31 ?type .
  ?item wdt:P840 ?place . # narration place is ?place
  ?place wdt:P17 ?country .
  ?country rdfs:label ?countrylabel FILTER (lang(?countrylabel) = "en") .

}
  GROUP BY ?countrylabel
  ORDER BY DESC(?count)

"""
query_result = mkwikidata.run_query(query, params={ })
results_df = pd.io.json.json_normalize(query_result['results']['bindings'])
results_df[['countrylabel.value', 'count.value']]

<ipython-input-22-143b73f42376>:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(query_result['results']['bindings'])


,countrylabel.value,count.value
0,United States of America,2303
1,United Kingdom,1374
2,France,676
3,Italy,624
4,Spain,560
...,...,...
307,Comoros,1
308,Habsburg monarchy,1
309,Lands of the Bohemian Crown,1
310,Transnistria,1


Let's try out the bubble chart visualization of [the query result](https://query.wikidata.org/#%23defaultView%3ABubbleChart%0ASELECT%20%3Fcountrylabel%20%28COUNT%28%3Fcountry%29%20AS%20%3Fcount%29%0AWHERE%0A%7B%0A%20%20VALUES%20%3Ftype%20%7Bwd%3AQ571%20wd%3AQ7725634%7D%20%20%23%20book%20or%20literary%20work%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%20.%0A%20%20%3Fitem%20wdt%3AP840%20%3Fplace%20.%20%23%20narration%20place%20is%20%3Fplace%0A%20%20%3Fplace%20wdt%3AP17%20%3Fcountry%20.%0A%20%20%3Fcountry%20rdfs%3Alabel%20%3Fcountrylabel%20FILTER%20%28lang%28%3Fcountrylabel%29%20%3D%20%22en%22%29%20.%0A%0A%7D%20%0A%20%20GROUP%20BY%20%3Fcountrylabel%0A%20%20ORDER%20BY%20DESC%28%3Fcount%29%0A) in Wikidata!

### Querying Wikidata with SPARQLWrapper ###

Alternatively, Wikidata KG and other KGs can be queried using the [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html) – a python interface for any SPARQL Endpoint.

For this, we have to install and import the following packages:


**SPARQLWrapper**

In [ ]:
!pip install -q sparqlwrapper    #install SPARQLwrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql") #determine SPARQL endpoint
sparql.setReturnFormat(JSON) #determine the output format

#### **Query 3**: Being a writer is dangerous! Let's see what is the most common cause of death among writers? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P106](https://www.wikidata.org/wiki/Property:P106), [wdt:509](https://www.wikidata.org/wiki/Property:P509), [wd:Q36180](https://www.wikidata.org/wiki/Q36180)


In [ ]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE
{
  ?person wdt:P106 wd:Q36180 . #occupation writer
  ?person wdt:P509 ?cause . #?cause is a death cause
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)
 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]

<ipython-input-25-c1d8d6c948a1>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


,causeLabel.value,count.value
0,cancer,1075
1,myocardial infarction,1035
2,tuberculosis,576
3,pneumonia,523
4,stroke,413
...,...,...
274,postoperative complications,2
275,brain congestion,2
276,coronavirus disease,2
277,November 2015 Paris attacks,2


#### **Query 4**: And if we talk about writers that died before 1800? ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P570](https://www.wikidata.org/wiki/Property:P570)

In [ ]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?person) AS ?count)
WHERE
{
  ?person wdt:P106 wd:Q36180; #ocupation writer
          wdt:P509 ?cause ; #?cause is a death cause
          wdt:P570 ?date . #death date
  FILTER (?date < "1800-01-01T00:00:00Z"^^xsd:dateTime) .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]

<ipython-input-26-a82016973f3d>:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


,causeLabel.value,count.value
0,decapitation,93
1,plague,63
2,tuberculosis,46
3,disease,44
4,death by burning,35
5,drowning,31
6,hanging,24
7,smallpox,23
8,stroke,22
9,malaria,17


So, being a writer before 1800 could often leave you without the head. :)

Explore more!



*   What was the worst year for writers? [List years](https://query.wikidata.org/#SELECT%20%3Fyear%20%28COUNT%28%3Fperson%29%20AS%20%3Fcount%29%0AWHERE%0A%7B%0A%20%20%3Fperson%20wdt%3AP106%20wd%3AQ36180%3B%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP509%20wd%3AQ204933%20%3B%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP570%20%3Fdate%20.%0A%20%20BIND%20%28YEAR%28%3Fdate%29%20AS%20%3Fyear%29%0A%7D%0AGROUP%20BY%20%3Fyear%0AORDER%20BY%20DESC%28%3Fcount%29%0A) in which the most writers were decapitated.
*   What is [the most dangerous month](https://query.wikidata.org/#SELECT%20%3Fmonth%20%28COUNT%28%3Fperson%29%20AS%20%3Fcount%29%0AWHERE%0A%7B%0A%20%20%3Fperson%20wdt%3AP106%20wd%3AQ36180%3B%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP509%20wd%3AQ468455%20%3B%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP570%20%3Fdate%20.%0A%20%20BIND%20%28MONTH%28%3Fdate%29%20AS%20%3Fmonth%29%0A%7D%0AGROUP%20BY%20%3Fmonth%0AORDER%20BY%20DESC%28%3Fcount%29%0A) for writers to be burnt?

*  [Show the images](https://query.wikidata.org/#%23defaultView%3AImageGrid%0ASELECT%20%3Fperson%20%3Fimage%0AWHERE%0A%7B%0A%20%20%3Fperson%20wdt%3AP106%20wd%3AQ36180%3B%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP509%20wd%3AQ12242142%20%3B%0AOPTIONAL%20%7B%20%3Fperson%20wdt%3AP18%20%3Fimage%20%7D%0A%7D%0A%0A) of writers who died from laughter.



*   Explore more and share your interesting queries and findings with us! :)


#### **Query 5**: Now let's see how writers tend to kill their characters. ####

Following additional Wikidata classes/instances and properties are required:

[wdt:P674](https://www.wikidata.org/wiki/Property:P674)

In [ ]:
sparql.setQuery("""
SELECT ?cause ?causeLabel (COUNT(?character) AS ?count)

WHERE
{
  VALUES ?type {wd:Q571 wd:Q7725634}  # book or literary work
  ?item wdt:P31 ?type .
  ?item wdt:P674 ?character .
  ?character wdt:P509 ?cause .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?cause ?causeLabel
HAVING(?count > 1)
ORDER BY DESC(?count)

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['causeLabel.value', 'count.value']]


#### **Query 6**: In the end, let's take a look at the authors that are still alive! List 100 youngest authors alive from the Eastern Europe. ####

In [ ]:
sparql.setQuery("""
SELECT DISTINCT ?personLabel ?countryLabel ?age
WHERE {
  ?person wdt:P106 wd:Q36180 ; #occupation writer
          wdt:P27 ?country ; #country of citizenship
          wdt:P569 ?birth_date.

  BIND(year(now()) - year(?birth_date) AS ?age)
  FILTER NOT EXISTS { ?person wdt:P570 ?death } #no death date for this person exist

  ?country wdt:P361 wd:Q27468 . #country is a part of Easter Europe

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en". }
 }

ORDER BY (?age)
LIMIT 100

 """
)
results = sparql.query().convert()   # execute SPARQL query and write result to "results"
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['personLabel.value', 'countryLabel.value', 'age.value']]

<ipython-input-27-3332d934c3a9>:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


,personLabel.value,countryLabel.value,age.value
0,Silva Souriau,Romania,NaN
1,Vojta Horák,Czech Republic,15
2,Tomáš Grygar,Czech Republic,17
3,Nela The Little Reporter,Poland,18
4,Vojtěch Sláma,Czech Republic,19
...,...,...,...
95,Ráchel Smékalová,Czech Republic,31
96,Markéta Damková,Czech Republic,31
97,Anna Basiková,Czech Republic,31
98,Dominik Fajkus,Czech Republic,31
